# Data Cleaning for PCA

In [1]:
import pandas as pd
import numpy as np

## Load + Basic Cleaning

In [2]:
path = "raw_data/ustc_historic_data.csv"
df = pd.read_csv(path, sep=";")

# parse timestamp and sort oldest -> newest
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)

# keep only the columns we need (optional but keeps things clean)
keep_cols = [c for c in [
    "timestamp", "open", "high", "low", "close", "volume", "marketCap", "circulatingSupply", "totalSupply"
] if c in df.columns]
df = df[keep_cols].copy()

# ensure numeric columns are numeric
num_cols = [c for c in df.columns if c != "timestamp"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# drop rows where core OHLC is missing
df = df.dropna(subset=[c for c in ["open", "high", "low", "close"] if c in df.columns]).reset_index(drop=True)

df.head()


,timestamp,open,high,low,close,volume,marketCap,circulatingSupply
0,2021-12-28 23:59:59.999000+00:00,1.000560,1.003111,0.999108,1.002239,1.395190e+08,1.005642e+10,1.003396e+10
1,2021-12-29 23:59:59.999000+00:00,1.002244,1.003347,0.999270,1.001423,1.575926e+08,1.007944e+10,1.006511e+10
2,2021-12-30 23:59:59.999000+00:00,1.001019,1.002950,0.999730,1.001984,1.106219e+08,1.011771e+10,1.009768e+10
3,2021-12-31 23:59:59.999000+00:00,1.001748,1.002327,0.998924,1.001440,1.357036e+08,1.013712e+10,1.012254e+10
4,2022-01-01 23:59:59.999000+00:00,1.001657,1.002204,0.998407,1.000757,1.266358e+08,1.013354e+10,1.012587e+10


## Rate of Change

In [3]:
# price percent changes
df["percent_change_24h"] = df["close"].pct_change(1) * 100
df["percent_change_7d"]  = df["close"].pct_change(7) * 100
df["percent_change_30d"] = df["close"].pct_change(30) * 100

# volume percent changes (only if volume exists)
if "volume" in df.columns:
    df["volume_percent_change_24h"] = df["volume"].pct_change(1) * 100
    df["volume_percent_change_7d"]  = df["volume"].pct_change(7) * 100
    df["volume_percent_change_30d"] = df["volume"].pct_change(30) * 100

## Market / Supply Log-Change

In [4]:
def ln_past_over_today(x, k):
    return np.log(x.shift(k) / x)

# market cap log-changes
if "marketCap" in df.columns:
    df["market_cap_percent_change_24h"] = ln_past_over_today(df["marketCap"], 1)
    df["market_cap_percent_change_7d"]  = ln_past_over_today(df["marketCap"], 7)
    df["market_cap_percent_change_30d"] = ln_past_over_today(df["marketCap"], 30)

# circulating supply log-changes
if "circulatingSupply" in df.columns:
    df["circulating_supply_percent_change_24h"] = ln_past_over_today(df["circulatingSupply"], 1)
    df["circulating_supply_percent_change_7d"]  = ln_past_over_today(df["circulatingSupply"], 7)
    df["circulating_supply_percent_change_30d"] = ln_past_over_today(df["circulatingSupply"], 30)

# total supply log-changes (your CSV likely doesn't have this; will run only if present)
if "totalSupply" in df.columns:
    df["total_supply_percent_change_24h"] = ln_past_over_today(df["totalSupply"], 1)
    df["total_supply_percent_change_7d"]  = ln_past_over_today(df["totalSupply"], 7)
    df["total_supply_percent_change_30d"] = ln_past_over_today(df["totalSupply"], 30)

## Volatility

In [5]:
# avoid log(0) issues
eps = 1e-12
O = df["open"].clip(lower=eps)
H = df["high"].clip(lower=eps)
L = df["low"].clip(lower=eps)
C = df["close"].clip(lower=eps)

df["realized_daily_volatility"] = np.sqrt(
    (np.log(H / O) * np.log(H / C)) + (np.log(L / O) * np.log(L / C))
)

## Price Deviation from $1

In [6]:
df["peg_error"] = df["close"] - 1
df["abs_peg_error"] = df["peg_error"].abs()

df["price_deviation_5d"]  = np.sqrt((df["peg_error"]**2).rolling(5).mean())
df["price_deviation_30d"] = np.sqrt((df["peg_error"]**2).rolling(30).mean())

downside = np.minimum(df["peg_error"], 0.0)
df["downward_price_deviation_5d"]  = np.sqrt((downside**2).rolling(5).mean())
df["downward_price_deviation_30d"] = np.sqrt((downside**2).rolling(30).mean())


## Final Cleanup

In [7]:
# replace inf/-inf from logs with NaN
df = df.replace([np.inf, -np.inf], np.nan)

# drop NaNs (ie for early rows due to rolling/shift)
df = df.dropna().reset_index(drop=True)
df.head()

,timestamp,open,high,low,close,volume,marketCap,circulatingSupply,percent_change_24h,percent_change_7d,...,circulating_supply_percent_change_24h,circulating_supply_percent_change_7d,circulating_supply_percent_change_30d,realized_daily_volatility,peg_error,abs_peg_error,price_deviation_5d,price_deviation_30d,downward_price_deviation_5d,downward_price_deviation_30d
0,2022-01-27 23:59:59.999000+00:00,1.000233,1.006147,0.995441,0.996486,8.794641e+08,1.121121e+10,1.125074e+10,-0.383843,-0.770964,...,-0.004692,-0.037807,-0.114459,0.007869,-0.003514,0.003514,0.001598,0.001794,0.001572,0.001355
1,2022-01-28 23:59:59.999000+00:00,0.996734,0.999967,0.988503,0.995149,2.709128e+09,1.121339e+10,1.126805e+10,-0.134154,-0.457796,...,-0.001537,-0.030648,-0.112896,0.008439,-0.004851,0.004851,0.002695,0.001984,0.002679,0.001619
2,2022-01-29 23:59:59.999000+00:00,0.995056,1.001707,0.995037,1.000395,7.285286e+08,1.126516e+10,1.126071e+10,0.527108,0.227126,...,0.000652,-0.023493,-0.109015,0.002971,0.000395,0.000395,0.002694,0.001952,0.002679,0.001619
3,2022-01-30 23:59:59.999000+00:00,1.000507,1.003632,0.996363,0.999787,2.988514e+08,1.126369e+10,1.126609e+10,-0.060697,-0.025268,...,-0.000477,-0.015300,-0.107032,0.005119,-0.000213,0.000213,0.002690,0.001934,0.002680,0.001619
4,2022-01-31 23:59:59.999000+00:00,0.999863,1.001065,0.995006,0.998031,3.330500e+08,1.124847e+10,1.127067e+10,-0.175691,-0.239455,...,-0.000406,-0.009321,-0.107110,0.004293,-0.001969,0.001969,0.002827,0.001963,0.002821,0.001659


## Combining Datasets

In [8]:
# create a daily date key (so merges are clean)
df["date"] = df["timestamp"].dt.date
df["date"] = pd.to_datetime(df["date"])

### Load Fear & Greed

In [9]:
fg = pd.read_csv("clean_data/cleaned_fear_greed_index.csv")

fg["date"] = pd.to_datetime(fg["date"], errors="coerce")  
fg = fg.dropna(subset=["date"]).sort_values("date")
fg["date"] = fg["date"].dt.tz_localize(None).dt.normalize()


### Load Fed Funds Rate

In [10]:
ff = pd.read_csv("clean_data/cleaned_fed_funds_rate.csv")
ff["date"] = pd.to_datetime(ff["date"], errors="coerce")
ff = ff.dropna(subset=["date"]).sort_values("date")
ff["date"] = ff["date"].dt.tz_localize(None).dt.normalize()

### Merge

In [11]:
print(df["date"].dtype)
print(fg["date"].dtype)
print(ff["date"].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]


In [12]:
final_df = df.merge(fg, on="date", how="left").merge(ff, on="date", how="left")
final_df = final_df.drop(columns=["date"])

In [13]:
# save
out_path = "clean_data/cleaned_data_for_pca.csv"
final_df.to_csv(out_path, index=False)

print("Saved:", out_path)
final_df.head(3)

Saved: clean_data/cleaned_data_for_pca.csv


,timestamp,open,high,low,close,volume,marketCap,circulatingSupply,percent_change_24h,percent_change_7d,...,realized_daily_volatility,peg_error,abs_peg_error,price_deviation_5d,price_deviation_30d,downward_price_deviation_5d,downward_price_deviation_30d,value,value_classification,fed_funds_rate
0,2022-01-27 23:59:59.999000+00:00,1.000233,1.006147,0.995441,0.996486,8.794641e+08,1.121121e+10,1.125074e+10,-0.383843,-0.770964,...,0.007869,-0.003514,0.003514,0.001598,0.001794,0.001572,0.001355,20,Extreme Fear,0.08
1,2022-01-28 23:59:59.999000+00:00,0.996734,0.999967,0.988503,0.995149,2.709128e+09,1.121339e+10,1.126805e+10,-0.134154,-0.457796,...,0.008439,-0.004851,0.004851,0.002695,0.001984,0.002679,0.001619,24,Extreme Fear,0.08
2,2022-01-29 23:59:59.999000+00:00,0.995056,1.001707,0.995037,1.000395,7.285286e+08,1.126516e+10,1.126071e+10,0.527108,0.227126,...,0.002971,0.000395,0.000395,0.002694,0.001952,0.002679,0.001619,24,Extreme Fear,0.08
